In [1]:
import tensorflow as tf
import numpy as np
import random
import cv2
import os
import time
import matplotlib.pyplot as plt

%matplotlib inline

from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers, metrics
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications.inception_v3 import preprocess_input


In [2]:
def create_triplets(directory, img_per_class=None):
    triplets = []
    folders = os.listdir(directory)
    try:
        folders.remove(".DS_Store")
    except ValueError:
        pass
    for folder in folders[:50]:

        path = os.path.join(directory, folder)
        files = os.listdir(path)
        try:
            files.remove(".DS_Store")
        except ValueError:
            pass
        

        # Creating training triplets
        for anchor_file in files[:img_per_class]:
            positive_file = random.choice([file for file in files if file != anchor_file])
            negative_folder = random.choice([f for f in folders[:10] if f != folder])
            negative_images = os.listdir(os.path.join(directory, negative_folder))
            try:
                negative_images.remove(".DS_Store")
            except ValueError:
                pass
            negative_file = random.choice(negative_images)
            triplets.append(((folder, anchor_file), (folder, positive_file), (negative_folder, negative_file)))

    random.shuffle(triplets)

    return triplets

In [3]:
# Setting random seeds to enable consistency while testing.
random.seed(5)
np.random.seed(5)
tf.random.set_seed(5)
ROOT = 'dataset'
def read_image(path):
    image_1 = cv2.imread(path)
    image = cv2.resize(image_1, (128, 128))

    if image is not None:
        return image
    else:
        print(f"Error: Unable to load image at path {path}")
        return None

In [4]:
def get_encoder(input_shape):
    """ Returns the image encoding model """

    pretrained_model = Xception(
        input_shape=input_shape,
        weights='imagenet',
        include_top=False,
        pooling='avg',
    )
    print(len(pretrained_model.layers))

    for i in range(len(pretrained_model.layers)-27):
        pretrained_model.layers[i].trainable = False

    encode_model = Sequential([
        pretrained_model,
        layers.Flatten(),
        layers.Dense(512, activation='relu', kernel_regularizer=l2(0.001)),
        layers.Dropout(0.5),

        layers.BatchNormalization(),
        layers.Dense(256, activation="relu", kernel_regularizer=l2(0.001)),
        layers.Dropout(0.5),

        layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))
    ], name="Encode_Model")

    print(len(encode_model.layers))
    return encode_model

In [5]:
class DistanceLayer(layers.Layer):
    # A layer to compute ‖f(A) - f(P)‖² and ‖f(A) - f(N)‖²
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, anchor, positive, negative):
        ap_distance = tf.reduce_sum(tf.square(anchor - positive), -1)
        an_distance = tf.reduce_sum(tf.square(anchor - negative), -1)
        return (ap_distance, an_distance)


def get_siamese_network(input_shape = (128, 128, 3)):
    encoder = get_encoder(input_shape)

    # Input Layers for the images
    anchor_input   = layers.Input(input_shape, name="Anchor_Input")
    positive_input = layers.Input(input_shape, name="Positive_Input")
    negative_input = layers.Input(input_shape, name="Negative_Input")

    ## Generate the encodings (feature vectors) for the images
    encoded_a = encoder(anchor_input)
    encoded_p = encoder(positive_input)
    encoded_n = encoder(negative_input)

    # A layer to compute ‖f(A) - f(P)‖² and ‖f(A) - f(N)‖²
    distances = DistanceLayer()(
        encoder(anchor_input),
        encoder(positive_input),
        encoder(negative_input)
    )

    # Creating the Model
    siamese_network = Model(
        inputs  = [anchor_input, positive_input, negative_input],
        outputs = distances,
        name = "Siamese_Network"
    )
    return siamese_network



In [6]:

class SiameseModel(Model):
    # Builds a Siamese model based on a base-model
    def __init__(self, siamese_network, margin=1.0):
        super(SiameseModel, self).__init__()

        self.margin = margin
        self.siamese_network = siamese_network
        self.loss_tracker = metrics.Mean(name="loss")

    def call(self, inputs):
        return self.siamese_network(inputs)

    def train_step(self, data):
        # GradientTape get the gradients when we compute loss, and uses them to update the weights
        with tf.GradientTape() as tape:
            loss = self._compute_loss(data)

        gradients = tape.gradient(loss, self.siamese_network.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.siamese_network.trainable_weights))
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def test_step(self, data):
        loss = self._compute_loss(data)

        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def _compute_loss(self, data):
        # Get the two distances from the network, then compute the triplet loss
        ap_distance, an_distance = self.siamese_network(data)
        loss = tf.maximum(ap_distance - an_distance + self.margin, 0.0)
        return loss

    @property
    def metrics(self):
        # We need to list our metrics so the reset_states() can be called automatically.
        return [self.loss_tracker]

In [7]:
def extract_encoder(model):
    encoder = get_encoder((128, 128, 3))
    i=0
    for e_layer in model.layers[0].layers[3].layers:
        layer_weight = e_layer.get_weights()
        encoder.layers[i].set_weights(layer_weight)
        i+=1
    return encoder


In [8]:
def get_custom_batch(triplet_list, data_type, batch_size=4, preprocess=True):
    batch_steps = len(triplet_list)//batch_size

    for i in range(batch_steps+1):
        anchor   = []
        positive = []
        negative = []

        j = i*batch_size
        while j<(i+1)*batch_size and j<len(triplet_list):
            a, p, n = triplet_list[j]
            # print(a,p,n)
            anchor.append(read_image(f'custom_ds/{data_type}/{a[0]}/{a[1]}'))
            positive.append(read_image(f'custom_ds/{data_type}/{p[0]}/{p[1]}'))
            negative.append(read_image(f'custom_ds/{data_type}/{n[0]}/{n[1]}'))
            j+=1

        anchor = np.array(anchor)
        positive = np.array(positive)
        negative = np.array(negative)

        if preprocess:
            anchor = preprocess_input(anchor)
            positive = preprocess_input(positive)
            negative = preprocess_input(negative)

        yield ([anchor, positive, negative])

In [10]:
def test_on_triplets(batch_size = 4):
    pos_scores, neg_scores = [], []

    for data in get_custom_batch(validation_triplet, 'validation',batch_size=batch_size):
        prediction = siamese_model.predict(data)
        # print(prediction)
        pos_scores += list(prediction[0])
        neg_scores += list(prediction[1])

    accuracy = np.sum(np.array(pos_scores) < np.array(neg_scores)) / len(pos_scores)
    ap_mean = np.mean(pos_scores)
    an_mean = np.mean(neg_scores)
    ap_stds = np.std(pos_scores)
    an_stds = np.std(neg_scores)

    print(f"Accuracy on validation = {accuracy:.5f}")
    return (accuracy, ap_mean, an_mean, ap_stds, an_stds)

In [9]:
# Example usage
train_triplet = create_triplets("custom_ds/train")
validation_triplet = create_triplets("custom_ds/validation")
test_triplet = create_triplets("custom_ds/test")
# validation_triplet = create_triplets("dataset/validation", 10)
# train_triplet.extend(validation_triplet)

print(len(train_triplet))
print(len(validation_triplet))

207
102


In [11]:
siamese_network = get_siamese_network()
siamese_network.summary()

133
8
Model: "Siamese_Network"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Anchor_Input (InputLayer)      [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Positive_Input (InputLayer)    [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Negative_Input (InputLayer)    [(None, 128, 128, 3  0           []                               
                                )]                                            

In [12]:
siamese_model = SiameseModel(siamese_network)

optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=0.001)

siamese_model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


# Path to the saved weights
weights_path = "second_training/siamese_best_acc_model_custom_best_acc"

# Load the previously saved weights
siamese_model.load_weights(weights_path)

print("Weights loaded successfully.")


best_acc_encoder = extract_encoder(siamese_model)
best_acc_encoder.save_weights("second_training/best_acc_encoder")
best_acc_encoder.summary()

Weights loaded successfully.
133
8
Model: "Encode_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 2048)              20861480  
                                                                 
 flatten_1 (Flatten)         (None, 2048)              0         
                                                                 
 dense_2 (Dense)             (None, 512)               1049088   
                                                                 
 dropout_2 (Dropout)         (None, 512)               0         
                                                                 
 batch_normalization_9 (Batc  (None, 512)              2048      
 hNormalization)                                                 
                                                                 
 dense_3 (Dense)             (None, 256)               131328    
                   

In [14]:
import math
def preprocess_image(image_path):
    image = cv2.imread(image_path)



    image1 = cv2.resize(image, (128, 128)) # Resize image
    image1 = np.expand_dims(image1, axis=0) # Add batch dimension
    image1 = preprocess_input(image1) # Assuming you have the preprocess_input function as used during training
    return image1

In [16]:
def extract_encoder(model):
    encoder = get_encoder((128, 128, 3))
    i=0
    for e_layer in model.layers[0].layers[3].layers:
        layer_weight = e_layer.get_weights()
        encoder.layers[i].set_weights(layer_weight)
        i+=1
    return encoder

encoder = extract_encoder(siamese_model)
encoder.save_weights("encoder")
encoder.summary()

133
8
Model: "Encode_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 2048)              20861480  
                                                                 
 flatten_2 (Flatten)         (None, 2048)              0         
                                                                 
 dense_4 (Dense)             (None, 512)               1049088   
                                                                 
 dropout_4 (Dropout)         (None, 512)               0         
                                                                 
 batch_normalization_14 (Bat  (None, 512)              2048      
 chNormalization)                                                
                                                                 
 dense_5 (Dense)             (None, 256)               131328    
                                                

In [26]:
validation_path = 'custom_ds/validation'
test_path = 'custom_ds/test'
folders = os.listdir(validation_path)


print(folders)

all_average_distances = {}

for class_folder in folders:

    class_path = os.path.join(validation_path, class_folder)
    files = os.listdir(class_path)
    all_average_distances[class_folder] = []

    for image in files:
        image_path1 = os.path.join(class_path, image)
        image1 = preprocess_image(image_path1)
        embedding1 = encoder.predict(image1)

        average_distances = {}

        for i, folder in enumerate(folders, start=0):

          test_class_path = os.path.join(test_path, folder)
          files = os.listdir(test_class_path)

          distances = []

          for j in range(5):
            random_file = random.choice(files)
            file_path = os.path.join( test_class_path, random_file)
            image2 = preprocess_image(file_path)

            embedding2 = encoder.predict(image2)
            distance = np.float32(np.linalg.norm(embedding1 - embedding2))
            distances.append(distance)

          # axs[i , 0].imshow(cv2.cvtColor(cv2.imread(image_path1), cv2.COLOR_BGR2RGB))
          # axs[i , 1].imshow(cv2.cvtColor(cv2.imread(file_path), cv2.COLOR_BGR2RGB))


          # Calculate the average distance for the current class
          average_distance = np.mean(distances)

          average_distances[folder] = average_distance
          # Add the average distance as a label on the corresponding image plot on the right
        print(f"{average_distances} for {class_folder}")

        all_average_distances[class_folder].append(average_distances)

        # # Determine if the minimum average distance is less than 1.5
        # if min(average_distances.values()) < 1.5:
        #     result = False
        # else:
        #     result = True

        # print(result)


['Thuvaraga', 'Thanoraj', 'Prannavi']
1/1 [==============================] - 0s 42ms/step
{'Thuvaraga': 0.21812999, 'Thanoraj': 1.4009248, 'Prannavi': 1.3481668} for Thuvaraga
1/1 [==============================] - 0s 39ms/step
{'Thuvaraga': 0.32262844, 'Thanoraj': 1.3987511, 'Prannavi': 1.3841916} for Thuvaraga
1/1 [==============================] - 0s 35ms/step
{'Thuvaraga': 0.2845845, 'Thanoraj': 1.3703184, 'Prannavi': 1.3896415} for Thuvaraga
1/1 [==============================] - 0s 36ms/step
{'Thuvaraga': 0.23923245, 'Thanoraj': 1.3840835, 'Prannavi': 1.4028976} for Thuvaraga
1/1 [==============================] - 0s 43ms/step
{'Thuvaraga': 0.133239, 'Thanoraj': 1.3870977, 'Prannavi': 1.3961961} for Thuvaraga
1/1 [==============================] - 0s 39ms/step
{'Thuvaraga': 0.16944678, 'Thanoraj': 1.3319126, 'Prannavi': 1.399593} for Thuvaraga
1/1 [==============================] - 0s 38ms/step
{'Thuvaraga': 0.21070524, 'Thanoraj': 1.3627132, 'Prannavi': 1.3838999} for Thuvaraga


In [27]:
import json
import numpy as np


print(all_average_distances)

with open("results.json", "w") as f:
    json.dump(all_average_distances, f, indent=4)

{'Thuvaraga': [{'Thuvaraga': 0.21812999, 'Thanoraj': 1.4009248, 'Prannavi': 1.3481668}, {'Thuvaraga': 0.32262844, 'Thanoraj': 1.3987511, 'Prannavi': 1.3841916}, {'Thuvaraga': 0.2845845, 'Thanoraj': 1.3703184, 'Prannavi': 1.3896415}, {'Thuvaraga': 0.23923245, 'Thanoraj': 1.3840835, 'Prannavi': 1.4028976}, {'Thuvaraga': 0.133239, 'Thanoraj': 1.3870977, 'Prannavi': 1.3961961}, {'Thuvaraga': 0.16944678, 'Thanoraj': 1.3319126, 'Prannavi': 1.399593}, {'Thuvaraga': 0.21070524, 'Thanoraj': 1.3627132, 'Prannavi': 1.3838999}, {'Thuvaraga': 0.35905042, 'Thanoraj': 1.3983595, 'Prannavi': 1.3738861}, {'Thuvaraga': 0.23597941, 'Thanoraj': 1.3904006, 'Prannavi': 1.3526535}, {'Thuvaraga': 0.3271948, 'Thanoraj': 1.3685166, 'Prannavi': 1.3890711}, {'Thuvaraga': 0.19381236, 'Thanoraj': 1.3560808, 'Prannavi': 1.400757}, {'Thuvaraga': 0.24822843, 'Thanoraj': 1.3653114, 'Prannavi': 1.4123409}, {'Thuvaraga': 0.15120678, 'Thanoraj': 1.3502766, 'Prannavi': 1.388562}, {'Thuvaraga': 0.53869015, 'Thanoraj': 1.361

TypeError: Object of type float32 is not JSON serializable